# 地理空間データの加工・可視化
- 地理空間データは、多くの市場で使用可能なデータとなっている。
    - 公的機関が一般公開していてとても有用
    - 様々なデータを組み合わせることが可能な応用力の高いデータ
## データ分析においては
- 商圏分析
- 適地選定
- 顧客分析
- BCP（事業継続計画: 防災・災害対策）
- 掃除ロボットの制御、紛失防止タグのような屋内測位技術

に用いられる。

## AI 予測であれば
- 感染症の感染予測マップ
- 不動産売買の適正価格予測

などに活用。

## 地理空間データの特徴や用語
### Shapefile
複数のファイルからなる GIS データのフォーマットの１つ。建物や道路の位置や形状、属性情報を持つベクターデータを格納するためのファイル形式。

### GIS（Geographic Information System）
地理情報システムの略称。地理空間データを総合的に管理・加工し、可視化や分析する仕組みを指す。
1. ラスターデータ
    - 地表を画像のように格子状に分割し、ピクセルごとに値を持っているデータ。
    - ピクセル座標を置換してピクセル１つ１つをテーブルデータとして扱う。
    - 実際の地形を画像として表現し、衛星写真や空中写真としての利用が多い
    - **１つ１つの地形や建造物に対して詳細で特徴を捉えたデータを扱うことが可能**
    - ピクセルの持つ情報量がデータ量に直結するため処理時間の問題、大規模な可視化による解析が難しい。
2. ベクターデータ
    - 点、線、ポリゴン（面）の３種の要素から構成。
        - 点: 位置情報
        - 線：経路情報
        - ポリゴン: 範囲情報
        に対応していると考えると理解しやすい。
    - 数的処理により表現するため、少ないデータ量で情報量の多い可視化を行うことができる
    - ピクセルではないため、スケールに依存せず、縮尺による情報の歪みが発生しない。
    - 連続的な色彩表現や、画像による詳細表現は苦手。
    - 地形図などは作成可能。